In [23]:
library(tidyverse)
library(targets)
library(tarchetypes)
library(Seurat)

In [17]:
other_clusters = qs::qread('../00_preprocessing/_targets/objects/exp_labelled_other') %>% `[[` %>% pull(labels) %>% unique
neuron_clusters = qs::qread('../00_preprocessing/_targets/objects/exp_RElabelled_neuron') %>% `[[` %>% pull(labels) %>% unique

In [18]:
other_clusters_tibble = tibble(
  label = other_clusters,
  object = rlang::syms(c("exp_labelled_other"))
)
neuron_clusters_tibble = tibble(
  label = neuron_clusters,
  object = rlang::syms(c("exp_labelled_neuron"))
)

clusters_tibble = rbind(other_clusters_tibble, neuron_clusters_tibble)
head(clusters_tibble)
tail(clusters_tibble)

label,object
<fct>,<list>
NG2_OPC2-Microglia,exp_labelled_other
NFOL,exp_labelled_other
MOL,exp_labelled_other
NG2_OPC2,exp_labelled_other
Astrocytes,exp_labelled_other
b2_Tany,exp_labelled_other


label,object
<fct>,<list>
Trh_Cxcl12,exp_labelled_neuron
Nfix_Htr2c,exp_labelled_neuron
Sst_Unc13c-Agrp,exp_labelled_neuron
Rgs16_Nmu,exp_labelled_neuron
Rgs16_Dlx1-Unassigned2,exp_labelled_neuron
Rgs16_Dlx1,exp_labelled_neuron


In [19]:
clusters_tibble$label = clusters_tibble %>% pull(label) %>% stringr::str_replace(fixed('-'), '__')

In [20]:
head(clusters_tibble)
tail(clusters_tibble)

label,object
<chr>,<list>
NG2_OPC2__Microglia,exp_labelled_other
NFOL,exp_labelled_other
MOL,exp_labelled_other
NG2_OPC2,exp_labelled_other
Astrocytes,exp_labelled_other
b2_Tany,exp_labelled_other


label,object
<chr>,<list>
Trh_Cxcl12,exp_labelled_neuron
Nfix_Htr2c,exp_labelled_neuron
Sst_Unc13c__Agrp,exp_labelled_neuron
Rgs16_Nmu,exp_labelled_neuron
Rgs16_Dlx1__Unassigned2,exp_labelled_neuron
Rgs16_Dlx1,exp_labelled_neuron


In [21]:
qs::qsave(clusters_tibble, 'clusters_tibble.qs')

In [22]:
clusters_tibble

label,object
<chr>,<list>
NG2_OPC2__Microglia,exp_labelled_other
NFOL,exp_labelled_other
MOL,exp_labelled_other
NG2_OPC2,exp_labelled_other
Astrocytes,exp_labelled_other
b2_Tany,exp_labelled_other
a2_Tanycytes,exp_labelled_other
MOL__a2_Tanycytes,exp_labelled_other
b1_Tanycytes,exp_labelled_other


In [39]:
combine_tibble = clusters_tibble %>% 
rowwise %>% 
mutate(genes_stat_obj = rlang::syms(c(paste0('genes_stat_pl_100_', label, collapse='')))) %>%
mutate(ctm_obj = rlang::syms(c(paste0('ctm_pl_', label, collapse='')))) %>%
mutate(lib_stat_obj = rlang::syms(c(paste0('lib_stat_pl_', label, collapse='')))) %>%
mutate(redundancy_stat_obj = rlang::syms(c(paste0('redundancy_stat_pl', label, collapse='')))) %>%
select(-object)
head(combine_tibble)
qs::qsave(combine_tibble, 'combine_tibble.qs')

label,genes_stat_obj,ctm_obj,lib_stat_obj,redundancy_stat_obj
<chr>,<list>,<list>,<list>,<list>
NG2_OPC2__Microglia,genes_stat_pl_100_NG2_OPC2__Microglia,ctm_pl_NG2_OPC2__Microglia,lib_stat_pl_NG2_OPC2__Microglia,redundancy_stat_plNG2_OPC2__Microglia
NFOL,genes_stat_pl_100_NFOL,ctm_pl_NFOL,lib_stat_pl_NFOL,redundancy_stat_plNFOL
MOL,genes_stat_pl_100_MOL,ctm_pl_MOL,lib_stat_pl_MOL,redundancy_stat_plMOL
NG2_OPC2,genes_stat_pl_100_NG2_OPC2,ctm_pl_NG2_OPC2,lib_stat_pl_NG2_OPC2,redundancy_stat_plNG2_OPC2
Astrocytes,genes_stat_pl_100_Astrocytes,ctm_pl_Astrocytes,lib_stat_pl_Astrocytes,redundancy_stat_plAstrocytes
b2_Tany,genes_stat_pl_100_b2_Tany,ctm_pl_b2_Tany,lib_stat_pl_b2_Tany,redundancy_stat_plb2_Tany


In [133]:
combine_tibble = qs::qread('combine_tibble.qs')
combine_targets_list = tar_map(
  values = combine_tibble,
  names = "label",
  tar_target(genes_stat_pl_100_00,
             genes_stat_obj %>%
             mutate(label = label))
)

In [149]:
combine_tibble = qs::qread('combine_tibble.qs')
combine_tibble

label,genes_stat_obj,ctm_obj,lib_stat_obj,redundancy_stat_obj
<chr>,<list>,<list>,<list>,<list>
NG2_OPC2__Microglia,genes_stat_pl_100_NG2_OPC2__Microglia,ctm_pl_NG2_OPC2__Microglia,lib_stat_pl_NG2_OPC2__Microglia,redundancy_stat_plNG2_OPC2__Microglia
NFOL,genes_stat_pl_100_NFOL,ctm_pl_NFOL,lib_stat_pl_NFOL,redundancy_stat_plNFOL
MOL,genes_stat_pl_100_MOL,ctm_pl_MOL,lib_stat_pl_MOL,redundancy_stat_plMOL
NG2_OPC2,genes_stat_pl_100_NG2_OPC2,ctm_pl_NG2_OPC2,lib_stat_pl_NG2_OPC2,redundancy_stat_plNG2_OPC2
Astrocytes,genes_stat_pl_100_Astrocytes,ctm_pl_Astrocytes,lib_stat_pl_Astrocytes,redundancy_stat_plAstrocytes
b2_Tany,genes_stat_pl_100_b2_Tany,ctm_pl_b2_Tany,lib_stat_pl_b2_Tany,redundancy_stat_plb2_Tany
a2_Tanycytes,genes_stat_pl_100_a2_Tanycytes,ctm_pl_a2_Tanycytes,lib_stat_pl_a2_Tanycytes,redundancy_stat_pla2_Tanycytes
MOL__a2_Tanycytes,genes_stat_pl_100_MOL__a2_Tanycytes,ctm_pl_MOL__a2_Tanycytes,lib_stat_pl_MOL__a2_Tanycytes,redundancy_stat_plMOL__a2_Tanycytes
b1_Tanycytes,genes_stat_pl_100_b1_Tanycytes,ctm_pl_b1_Tanycytes,lib_stat_pl_b1_Tanycytes,redundancy_stat_plb1_Tanycytes
